In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.X_train, self.y_train), (self.X_test, self.y_test) = fashion_mnist.load_data()
        self.X_train = np.expand_dims(self.X_train.astype(np.float32)/255.0,axis=-1)
        self.X_test = np.expand_dims(self.X_test.astype(np.float32)/255.0,axis=-1)
        self.y_train = self.y_train.astype(np.int32)
        self.y_test = self.y_test.astype(np.int32)
        self.num_train, self.num_test = self.X_train.shape[0], self.X_test.shape[0]

    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.num_train, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_train[index],224,224,)
        return resized_images.numpy(), self.y_train[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.num_test, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_test[index],224,224,)
        return resized_images.numpy(), self.y_test[index]

batch_size = 128
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)


x_batch shape: (128, 224, 224, 1) y_batch shape: (128,)


In [4]:
def vgg_block(num_convs, num_channels):
    blk = keras.Sequential()
    for _ in range(num_convs):
        blk.add(
            keras.layers.Conv2D(
                num_channels, kernel_size=3, 
                padding='same', activation='relu'
            )
        )
    blk.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
    return blk

In [6]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

def vgg(conv_arch):
    net = keras.Sequential()
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    net.add(keras.Sequential([
        layers.Flatten(), 
        layers.Dense(4096, activation='relu'), 
        layers.Dropout(0.5), 
        layers.Dense(4096, activation='relu'), 
        layers.Dropout(0.5), 
        layers.Dense(10, activation='sigmoid')
    ]))
    return net

net = vgg(conv_arch)

In [7]:
X = tf.random.uniform([1, 224, 224, 1])
for blk in net.layers:
    X = blk(X)
    print(blk.name, X.shape)

sequential_1 (1, 112, 112, 64)
sequential_2 (1, 56, 56, 128)
sequential_3 (1, 28, 28, 256)
sequential_4 (1, 14, 14, 512)
sequential_5 (1, 7, 7, 512)
sequential_6 (1, 10)


In [8]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [10]:
def train_vgg():
    epochs = 5
    num_iter = dataLoader.num_train // batch_size
    for ep in range(epochs):
        for n in range(num_iter):
            X_batch, y_batch = dataLoader.get_batch_train(batch_size)
            net.fit(X_batch, y_batch)
            if n % 20 == 0:
                net.save_weights('vgg_weights.h5')

net.compile(
    optimizer = keras.optimizers.SGD(
        learning_rate=0.05, momentum=0.0, nesterov=False
    ), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

train_vgg()

Train on 128 samples
128/128 [==============================] - 2s 13ms/sample - loss: 2.3027 - accuracy: 0.0703
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.3018 - accuracy: 0.1641
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.3016 - accuracy: 0.1484
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.3012 - accuracy: 0.2031
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2999 - accuracy: 0.2344
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.3004 - accuracy: 0.1875
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2997 - accuracy: 0.2344
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2974 - accuracy: 0.2188
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2973 - accuracy

128/128 [==============================] - 0s 3ms/sample - loss: 0.9538 - accuracy: 0.6094
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 1.1003 - accuracy: 0.5391
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.9761 - accuracy: 0.5625
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.9533 - accuracy: 0.6406
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.9581 - accuracy: 0.6094
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.8911 - accuracy: 0.6562
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 1.0071 - accuracy: 0.5859
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.9365 - accuracy: 0.5156
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.8436 - accuracy: 0.6797
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 0.6767 - accuracy: 0.7344
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.6922 - accuracy: 0.8047
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.6540 - accuracy: 0.7656
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.6464 - accuracy: 0.7344
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.6359 - accuracy: 0.7656
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5572 - accuracy: 0.7734
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.7509 - accuracy: 0.7188
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.6476 - accuracy: 0.7266
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5770 - accuracy: 0.7812
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 0.4433 - accuracy: 0.8125
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.4383 - accuracy: 0.8203
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5426 - accuracy: 0.7969
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5362 - accuracy: 0.7578
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.3067 - accuracy: 0.8672
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.6672 - accuracy: 0.7500
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.6209 - accuracy: 0.7891
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.4821 - accuracy: 0.8047
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5377 - accuracy: 0.7891
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 0.2731 - accuracy: 0.8906
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.3918 - accuracy: 0.8750
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5129 - accuracy: 0.8125
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2495 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.4752 - accuracy: 0.8281
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.5072 - accuracy: 0.8047
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.3547 - accuracy: 0.8359
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.4708 - accuracy: 0.8047
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 0.3263 - accuracy: 0.8906
Train on 128 

...

128/128 [==============================] - 0s 3ms/sample - loss: 0.1190 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1638 - accuracy: 0.9375
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2378 - accuracy: 0.9453
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1544 - accuracy: 0.9375
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2167 - accuracy: 0.9453
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1868 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2039 - accuracy: 0.9375
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1671 - accuracy: 0.9219
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2007 - accuracy: 0.9297
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 0.1123 - accuracy: 0.9688
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1966 - accuracy: 0.9219
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2076 - accuracy: 0.9219
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1269 - accuracy: 0.9609
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1515 - accuracy: 0.9453
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.0977 - accuracy: 0.9688
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2561 - accuracy: 0.9219
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2474 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2361 - accuracy: 0.9375
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 0.1892 - accuracy: 0.9141
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1472 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2606 - accuracy: 0.9062
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1693 - accuracy: 0.9453
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1352 - accuracy: 0.9609
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1421 - accuracy: 0.9453
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2353 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.2901 - accuracy: 0.8750
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1767 - accuracy: 0.9453
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 0.1970 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 1s 4ms/sample - loss: 0.1202 - accuracy: 0.9531
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1828 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1985 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1821 - accuracy: 0.9375
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1205 - accuracy: 0.9609
Train on 128 samples
128/128 [==============================] - 0s 4ms/sample - loss: 0.2248 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1162 - accuracy: 0.9453
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1860 - accuracy: 0.9531
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 0.1772 - accuracy: 0.9141
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1542 - accuracy: 0.9297
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1187 - accuracy: 0.9688
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 0.1268 - accuracy: 0.9609


In [12]:
net.load_weights("vgg_weights.h5")

x_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(x_test, y_test, verbose=2)

2000/2000 - 2s - loss: 0.2334 - accuracy: 0.9195


[0.2334369202554226, 0.9195]